# Mobility Data from census.gov API

In [2]:
import pandas as pd
import requests

In [59]:
front='https://api.census.gov/data/'
back='/acs/acs1/pums?tabulate=weight(PWGTP)&col+JWTR&row+ucgid&ucgid=0400000US01,\
0400000US02,0400000US04,0400000US05,0400000US06,0400000US08,0400000US09,0400000US10,0400000US11,0400000US12,0400000US13,\
0400000US15,0400000US16,0400000US17,0400000US18,0400000US19,0400000US20,0400000US21,0400000US22,0400000US23,0400000US24,\
0400000US25,0400000US26,0400000US27,0400000US28,0400000US29,0400000US30,0400000US31,0400000US32,0400000US33,0400000US34,\
0400000US35,0400000US36,0400000US37,0400000US38,0400000US39,0400000US40,0400000US41,0400000US42,0400000US44,0400000US45,\
0400000US46,0400000US47,0400000US48,0400000US49,0400000US50,0400000US51,0400000US53,0400000US54,0400000US55,0400000US56'

In [64]:
#get all links
urldict={}
for year in range(2004,2019):
    url=front+str(year)+back
    urldict[year]=url

#2019's link a little different: JWTR becomes JWTRNS
urldict[2019]='https://api.census.gov/data/2019\
/acs/acs1/pums?tabulate=weight(PWGTP)&col+JWTRNS&row+ucgid&ucgid=0400000US01,\
0400000US02,0400000US04,0400000US05,0400000US06,0400000US08,0400000US09,0400000US10,0400000US11,0400000US12,0400000US13,\
0400000US15,0400000US16,0400000US17,0400000US18,0400000US19,0400000US20,0400000US21,0400000US22,0400000US23,0400000US24,\
0400000US25,0400000US26,0400000US27,0400000US28,0400000US29,0400000US30,0400000US31,0400000US32,0400000US33,0400000US34,\
0400000US35,0400000US36,0400000US37,0400000US38,0400000US39,0400000US40,0400000US41,0400000US42,0400000US44,0400000US45,\
0400000US46,0400000US47,0400000US48,0400000US49,0400000US50,0400000US51,0400000US53,0400000US54,0400000US55,0400000US56'

In [115]:
#dict for renaming cols
renamecols={'JWTR8':"Motorcycle",
 'JWTR6':"Ferry boat",
 'JWTR10':"Walked",
 'JWTR0':"Missing",
 'JWTR7':"Taxicab",
 'JWTR1':"Car/truck/van",
 'JWTR3':"Streetcar",
 'JWTR2':"Bus",
 'JWTR12':"Other",
 'JWTR11':"Worked At Home",
 'JWTR9':"Bicycle",
 'JWTR4':"Subway",
 'JWTR5':"Railroad",
 'ucgid':"ucgid"}
finalcols=list(renamecols.values())

renamecols2019={'JWTRNS08':"Motorcycle",
 'JWTRNS06':"Ferry boat",
 'JWTRNS10':"Walked",
 'JWTRNS0':"Missing",
 'JWTRNS07':"Taxicab",
 'JWTRNS01':"Car/truck/van",
 'JWTRNS03':"Streetcar",
 'JWTRNS02':"Bus",
 'JWTRNS12':"Other",
 'JWTRNS11':"Worked At Home",
 'JWTRNS09':"Bicycle",
 'JWTRNS04':"Subway",
 'JWTRNS05':"Railroad",
 'ucgid':"ucgid"}

In [105]:
urldict.keys()[-1]

TypeError: 'dict_keys' object is not subscriptable

In [149]:
#get data
finaldf= pd.DataFrame(columns=['Car/truck/van','Car_pct','Worked At Home','WFH_pct','Year','ucgid'])

for year in list(urldict.keys()):
    
    print(year)
    
    url=urldict[year]
    response = requests.get(url)
    data=response.json()
    
    #header
    cols=[]
    for i in data[0][:-1]:
        col=list(i.keys())[0]+list(i.values())[0]
        cols.append(col)
    cols.append(data[0][-1])
    
    df=pd.DataFrame(data[1:],columns=cols)
    if year==2019:
        df = df.rename(columns = renamecols2019)
    else:
        df = df.rename(columns = renamecols)
    df['WFH_pct'] = df['Worked At Home']/(df.drop('ucgid', axis=1).sum(axis=1))
    df['Car_pct'] = df['Car/truck/van']/(df.drop('ucgid', axis=1).sum(axis=1))
    df['Year']=int(year)
    df=df.loc[:,['Car/truck/van','Car_pct','Worked At Home','WFH_pct','Year','ucgid']]
    
    finaldf=pd.concat([finaldf,df], axis=0, ignore_index=True)

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [150]:
finaldf

,Car/truck/van,Car_pct,Worked At Home,WFH_pct,Year,ucgid
0,1835719,0.415833,44526,0.010086,2004,0400000US01
1,243550,0.382963,13859,0.021792,2004,0400000US02
2,2195828,0.389445,92132,0.016340,2004,0400000US04
3,1109112,0.415651,27844,0.010435,2004,0400000US05
4,13387756,0.381881,684711,0.019531,2004,0400000US06
...,...,...,...,...,...,...
811,3672166,0.430222,256477,0.030048,2019,0400000US51
812,3024275,0.397153,240429,0.031574,2019,0400000US53
813,666556,0.371932,28433,0.015865,2019,0400000US54
814,2628605,0.451462,146411,0.025146,2019,0400000US55


### Convert geoid to State

In [123]:
geoiddf = pd.read_csv('https://raw.githubusercontent.com/camillol/cs424p3/master/data/Population-Density%20By%20County.csv')

In [131]:
geoiddf=geoiddf[['GEO.id','GEO.display-label']].drop_duplicates().reset_index(drop=True)

In [133]:
geoiddf.columns = ['ucgid', 'State']

In [155]:
geoiddf.head()

,ucgid,State
0,0400000US01,Alabama
1,0400000US02,Alaska
2,0400000US04,Arizona
3,0400000US05,Arkansas
4,0400000US06,California


In [152]:
finaldf=pd.merge(finaldf, geoiddf, on='ucgid', how='outer')

In [153]:
finaldf.to_csv('censusgovworkplace.csv',index=False)

# WFH data from census.gov pulse survey
https://www.census.gov/programs-surveys/household-pulse-survey/data.html

In [158]:
import requests, bs4, re
from bs4 import BeautifulSoup as bs


In [159]:
url = 'https://www.census.gov/programs-surveys/household-pulse-survey/data.html'
#Yearlinks.append(year)
response = requests.get(url)
status = response.status_code
page = response.text
soup = bs(page)

In [160]:
weeks = soup.findAll('p', text = re.compile('Week'), attrs = {'class' : 'uscb-margin-TB-02 uscb-title-3'})

In [255]:
weekinfo={}
for week in weeks:
    week_num=week.text.split()[0]+" "+week.text.split()[1]
    num=int(week.text.split()[1])
    day=week.text.strip().split(':')[1].strip().split('–')[0].strip()
    if num < 22:
        year=2020
    elif num < 41:
        year=2021
    else:
        year=2022
    weekinfo[num]={}
    weekinfo[num]['week_num']=week_num
    weekinfo[num]['year']=year
    weekinfo[num]['day']=day


In [186]:
#pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [252]:
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [380]:
finaldata=[]

for num in range(13,34):
    
    
    
    year=weekinfo[num]['year']
    day=weekinfo[num]['day']
    
    print(num)
    print(year)
    print(day)
    
    link='https://www2.census.gov/programs-surveys/demo/tables/hhp/'+str(year)+'/wk'+str(num)+'/transport1_week'+str(num)+'.xlsx'
    
    
    
    xls = pd.ExcelFile(link)
    for state in states:
        
        mydict={}

        data = pd.read_excel(xls,state)
        
        new_header = data.iloc[3] #grab the first row for the header
        data = data[4:] #take the data less the header row
        data.columns = new_header #set the header row as the df header
        data=data.replace('-',0)
        
        if num < 22:
            wftpct=(data.iloc[2,2])/(data.iloc[2,2]+data.iloc[2,3]+data.iloc[2,4])

            age1824pct=(data.iloc[4,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])
            age2539pct=(data.iloc[5,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])
            age4054pct=(data.iloc[6,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])
            age55pluspct=(data.iloc[7,2]+data.iloc[8,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])

            income50minuspct=(data.iloc[48,2]+data.iloc[49,2]+data.iloc[50,2])/(data.iloc[48,2]+data.iloc[49,2]+data.iloc[50,2]+data.iloc[51,2]\
                                  +data.iloc[52,2]+data.iloc[53,2]+data.iloc[54,2]+data.iloc[55,2])
            income50100pct=(data.iloc[51,2]+data.iloc[52,2])/(data.iloc[48,2]+data.iloc[49,2]+data.iloc[50,2]+data.iloc[51,2]\
                                          +data.iloc[52,2]+data.iloc[53,2]+data.iloc[54,2]+data.iloc[55,2])
            income100pluspct=(data.iloc[53,2]+data.iloc[54,2]+data.iloc[55,2])/(data.iloc[48,2]+data.iloc[49,2]+data.iloc[50,2]+data.iloc[51,2]\
                                          +data.iloc[52,2]+data.iloc[53,2]+data.iloc[54,2]+data.iloc[55,2])
        
        elif num < 28:
            wftpct=(data.iloc[2,2])/(data.iloc[2,2]+data.iloc[2,3]+data.iloc[2,4])

            age1824pct=(data.iloc[4,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])
            age2539pct=(data.iloc[5,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])
            age4054pct=(data.iloc[6,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])
            age55pluspct=(data.iloc[7,2]+data.iloc[8,2])/(data.iloc[4,2]+data.iloc[5,2]+data.iloc[6,2]+data.iloc[7,2]+data.iloc[8,2])

            income50minuspct=(data.iloc[41,2]+data.iloc[42,2]+data.iloc[43,2])/(data.iloc[41,2]+data.iloc[42,2]+data.iloc[43,2]+data.iloc[44,2]\
                                  +data.iloc[45,2]+data.iloc[46,2]+data.iloc[47,2]+data.iloc[48,2])
            income50100pct=(data.iloc[44,2]+data.iloc[45,2])/(data.iloc[41,2]+data.iloc[42,2]+data.iloc[43,2]+data.iloc[44,2]\
                                  +data.iloc[45,2]+data.iloc[46,2]+data.iloc[47,2]+data.iloc[48,2])
            income100pluspct=(data.iloc[46,2]+data.iloc[47,2]+data.iloc[48,2])/(data.iloc[41,2]+data.iloc[42,2]+data.iloc[43,2]+data.iloc[44,2]\
                                  +data.iloc[45,2]+data.iloc[46,2]+data.iloc[47,2]+data.iloc[48,2])          
        
        else:
            #print(state)
            #print(data.iloc[3,2])
            
            
            wftpct=(data.iloc[3,2]+data.iloc[3,3]+data.iloc[3,4])/(data.iloc[3,2]+data.iloc[3,3]+data.iloc[3,4]+data.iloc[3,5])
            
            agesum=(data.iloc[5,2]+data.iloc[5,3]+data.iloc[6,2]+data.iloc[6,3]+data.iloc[7,2]+data.iloc[7,3]\
                                            +data.iloc[8,2]+data.iloc[8,3]+data.iloc[9,2]+data.iloc[9,3])
            age1824pct=(data.iloc[5,2]+data.iloc[5,3])/agesum
            age2539pct=(data.iloc[6,2]+data.iloc[6,3])/agesum
            age4054pct=(data.iloc[7,2]+data.iloc[7,3])/agesum
            age55pluspct=(data.iloc[8,2]+data.iloc[8,3]+data.iloc[9,2]+data.iloc[9,3])/agesum
            
            incomesum=data.iloc[50,2]+data.iloc[50,3]+data.iloc[51,2]+data.iloc[51,3]+data.iloc[52,2]+data.iloc[52,3]\
                        +data.iloc[53,2]+data.iloc[53,3]+data.iloc[54,2]+data.iloc[54,3]\
                        +data.iloc[55,2]+data.iloc[55,3]+data.iloc[56,2]+data.iloc[56,3]+data.iloc[57,2]+data.iloc[57,3]

            income50minuspct=(data.iloc[50,2]+data.iloc[50,3]+data.iloc[51,2]+data.iloc[51,3]+data.iloc[52,2]+data.iloc[52,3])/incomesum
            income50100pct=(data.iloc[53,2]+data.iloc[53,3]+data.iloc[54,2]+data.iloc[54,3])/incomesum
            income100pluspct=(data.iloc[55,2]+data.iloc[55,3]+data.iloc[56,2]+data.iloc[56,3]+data.iloc[57,2]+data.iloc[57,3])/incomesum
        
        mydict['year']=year
        mydict['day']=day
        mydict['state']=state
        mydict['wfhpct']=wftpct
        mydict['age1824pct']=age1824pct
        mydict['age2539pct']=age2539pct
        mydict['age4054pct']=age4054pct
        mydict['age55pluspct']=age55pluspct
        mydict['income50minuspct']=income50minuspct
        mydict['income50100pct']=income50100pct
        mydict['income100pluspct']=income100pluspct
        
        finaldata.append(mydict)

13
2020
August 19
14
2020
September 2
15
2020
September 16
16
2020
September 30
17
2020
October 14
18
2020
October 28
19
2020
November 11
20
2020
November 25
21
2020
December 9
22
2021
January 6
23
2021
January 20
24
2021
February 3
25
2021
February 17
26
2021
March 3
27
2021
March 17
28
2021
April 14
29
2021
April 28
30
2021
May 12
31
2021
May 26
32
2021
June 9
33
2021
June 23


In [381]:
finaldfpulse=pd.DataFrame(finaldata)

In [382]:
finaldfpulse['year']=finaldfpulse['year'].astype(str)

In [383]:
finaldfpulse['date']=finaldfpulse['day']+" "+finaldfpulse['year']

In [384]:
finaldfpulse

,year,day,state,wfhpct,age1824pct,age2539pct,age4054pct,age55pluspct,income50minuspct,income50100pct,income100pluspct,date
0,2020,August 19,AK,0.328396,0.117912,0.354686,0.314123,0.213279,0.179566,0.341478,0.478956,August 19 2020
1,2020,August 19,AL,0.251539,0.124787,0.347698,0.272252,0.255263,0.176606,0.393772,0.429622,August 19 2020
2,2020,August 19,AR,0.263968,0.217166,0.313468,0.253318,0.216048,0.193305,0.369093,0.437602,August 19 2020
3,2020,August 19,AZ,0.349531,0.113138,0.328943,0.299633,0.258287,0.207135,0.346091,0.446774,August 19 2020
4,2020,August 19,CA,0.397054,0.108920,0.342453,0.284702,0.263926,0.142033,0.275792,0.582175,August 19 2020
...,...,...,...,...,...,...,...,...,...,...,...,...
1066,2021,June 23,VT,0.355129,0.104608,0.202299,0.360099,0.332994,0.193933,0.297414,0.508653,June 23 2021
1067,2021,June 23,WA,0.364828,0.078438,0.367913,0.284143,0.269505,0.131542,0.238888,0.629570,June 23 2021
1068,2021,June 23,WI,0.267081,0.082114,0.287553,0.329563,0.300771,0.139759,0.372068,0.488173,June 23 2021
1069,2021,June 23,WV,0.196620,0.139326,0.256338,0.340609,0.263727,0.093350,0.374186,0.532464,June 23 2021


In [385]:
finaldfpulse.to_csv('censusgovpulse_fixed.csv',index=False)